Importing Required Libraries

In [22]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



Load the Dataset

In [23]:
file_path = "Data/coin_Bitcoin.csv"
df = pd.read_csv(file_path)

Data Preprocessing

In [24]:
df["Date"] = pd.to_datetime(df["Date"], errors='coerce')
df = df.sort_values("Date")
df = df.dropna()

features = ["High", "Low", "Open", "Volume", "Marketcap"]
target = "Close"

Normalize the data using MinMaxScaler

In [25]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[features + [target]])

Prepare Sequences for LSTM Model

In [26]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :-1]) 
        y.append(data[i+seq_length, -1])  
    return np.array(X), np.array(y)

seq_length = 10 
X, y = create_sequences(df_scaled, seq_length)



Split data into training and testing sets

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


Build the LSTM Model

In [28]:
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(seq_length, X.shape[2])),
    LSTM(50),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)
])

/Users/tejaskalyankar/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Compile the model

In [29]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

Train the model

In [30]:
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test), verbose=1)

Epoch 1/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 8.4513e-04 - mae: 0.0170 - val_loss: 0.0055 - val_mae: 0.0411
Epoch 2/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.9923e-04 - mae: 0.0080 - val_loss: 0.0054 - val_mae: 0.0402
Epoch 3/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.3567e-04 - mae: 0.0061 - val_loss: 0.0052 - val_mae: 0.0408
Epoch 4/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 9.5779e-05 - mae: 0.0058 - val_loss: 0.0025 - val_mae: 0.0285
Epoch 5/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 9.1577e-05 - mae: 0.0058 - val_loss: 0.0033 - val_mae: 0.0323
Epoch 6/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 9.2092e-05 - mae: 0.0056 - val_loss: 0.0011 - val_mae: 0.0205
Epoch 7/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.1432e-04 - mae: 0.0064 - val_loss: 0.0011 - val_mae: 0.0210
Epoch 8/20
149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 8.7434e-05 - mae: 0.0059 - val_loss: 8.9969e-04 - val_mae: 0.0180
Epoch 9/20
149/149 

Evaluate the Model

In [31]:
loss, mae = model.evaluate(X_test, y_test)
print(f"Model Loss: {loss:.4f}, Model MAE: {mae:.4f}")

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.6347e-04 - mae: 0.0083 
Model Loss: 0.0008, Model MAE: 0.0172


Predictions and Evaluation

In [32]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-Squared (R2 Score): {r2:.4f}")

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Mean Squared Error (MSE): 0.0008
Mean Absolute Error (MAE): 0.0172
R-Squared (R2 Score): 0.9892
